In [2]:
import numpy as np 
import pandas as pd 

In [ ]:
#load raw data in dataframe
df = pd.read_csv("Bitcoin_tweets.csv")

C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2352257836.py:1: DtypeWarning: Columns (4,5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Bitcoin_tweets.csv")


In [ ]:
df.shape

(3840626, 13)

In [ ]:
# drop columns not required for further processing
df.drop(['user_name','user_location','user_description','user_created','user_friends','user_favourites','user_verified','source','is_retweet'],axis=1,inplace=True)

In [ ]:
df.head()

,user_followers,date,text,hashtags
0,8534.0,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin']
1,6769.0,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']"
2,128.0,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN
3,625.0,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']"
4,1249.0,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC']


In [ ]:
# remove hashtags most likely to indiacte to tweets which can be considered noise
df = df[df['hashtags'].str.contains('Giveaway')==False]
df = df[df['hashtags'].str.contains('Cashback')==False]
df = df[df['hashtags'].str.contains('Airdrop')==False]
df = df[df['hashtags'].str.contains('nft')==False]
df = df[df['hashtags'].str.contains('giveaway')==False]
df = df[df['hashtags'].str.contains('giveaway')==False]

In [ ]:
df.shape

(3380692, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3380692 entries, 0 to 3840625
Data columns (total 4 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   user_followers  object
 1   date            object
 2   text            object
 3   hashtags        object
dtypes: object(4)
memory usage: 129.0+ MB


In [ ]:
df['date'] =  pd.to_datetime(df['date'],errors='coerce')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3380692 entries, 0 to 3840625
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_followers  object        
 1   date            datetime64[ns]
 2   text            object        
 3   hashtags        object        
dtypes: datetime64[ns](1), object(3)
memory usage: 129.0+ MB


In [ ]:
df.dropna(inplace=True)

In [ ]:
#limiting number of tweets based on dates as BERT takes a long time to run
start_date = '2021-04-01'
end_date = '2021-04-30'
# Select DataFrame rows between two dates
mask = (df['date'] > start_date) & (df['date'] <= end_date)
df2 = df.loc[mask]

In [ ]:
df2.head()

,user_followers,date,text,hashtags
1999534,511.0,2022-01-14 23:59:59,Death Cross #bitcoin dump? https://t.co/UPjFVs...,['bitcoin']
1999535,26.0,2022-01-14 23:59:56,#bitcoin is revolutionizing the insurance indu...,['bitcoin']
1999536,94.0,2022-01-14 23:59:56,Teaser! #Bitcoin #cryptocurrency #furniture ht...,"['Bitcoin', 'cryptocurrency', 'furniture']"
1999537,51.0,2022-01-14 23:59:56,I am claiming my free Lightning sats from @_bi...,"['Bitcoin', 'BTC', 'LN', 'LightningNetwork', '..."
1999541,7624.0,2022-01-14 23:59:44,#SCRT Did As We Expected . \n\n#BTC #BITCOIN $...,"['SCRT', 'BTC', 'BITCOIN']"


In [ ]:
df2.shape

(552004, 4)

In [ ]:
df2['user_followers'] = df['user_followers'].astype(float)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 552004 entries, 1999534 to 2830294
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_followers  552004 non-null  float64       
 1   date            552004 non-null  datetime64[ns]
 2   text            552004 non-null  object        
 3   hashtags        552004 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 21.1+ MB


C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\1877016129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['user_followers'] = df['user_followers'].astype(float)


In [ ]:
# remove tweets from accounts with less than 11 followers as they are likely to have less impact on genearl market sentiment
df2 = df2[df2.user_followers > 10]
df2.tail()

,user_followers,date,text,hashtags
2830288,9175.0,2022-04-13 11:49:16,Shiba Inu and 9 Other Cryptocurrencies to buy ...,"['Cryptocurrencies', 'ShibaInu', 'Robinhood', ..."
2830289,519.0,2022-04-13 11:49:15,#Bitcoin Last Price $39840 #BTC 🚀\nDaily Indic...,"['Bitcoin', 'BTC', 'Ethereum', 'ETH', 'Crypto'..."
2830292,82.0,2022-04-13 11:49:11,@YoungDeanz @Vikingobitcoin2 My time preferenc...,['BTC']
2830293,24.0,2022-04-13 11:49:09,Legacy markets open flat? #bitcoin was flat?\n...,"['bitcoin', 'btc', 'crypto']"
2830294,78.0,2022-04-13 11:49:08,"#BTC.4 hour level, the lower support range of ...",['BTC']


In [ ]:
df2.shape

(505834, 4)

In [ ]:
import re

In [ ]:
# data preprocessing to remove urls, hashtags, mentions, special characters
df2['text'] = df2['text'].str.lower()
df2['text'] = df2['text'].str.replace("@[A-Za-z0-9_]+","")
df2['text'] = df2['text'].str.replace("#[A-Za-z0-9_]+","")
df2['text'] = df2['text'].str.replace(r"http\S+", "")
df2['text'] = df2['text'].str.replace(r"www.\S+", "")
df2['text'] = df2['text'].str.replace('[()!?]', ' ')
df2['text'] = df2['text'].str.replace('\[.*?\]',' ')
df2['text'] = df2['text'].str.replace("[^a-z0-9]"," ")
df2.head()

C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2132609368.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['text'] = df2['text'].str.replace("@[A-Za-z0-9_]+","")
C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2132609368.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['text'] = df2['text'].str.replace("#[A-Za-z0-9_]+","")
C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2132609368.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['text'] = df2['text'].str.replace(r"http\S+", "")
C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2132609368.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['text'] = df2['text'].str.replace(r"www.\S+", "")
C:\Users\vipul\AppData\Local\Temp\ipykernel_29104\2132609368.py:6: FutureWarning: The default value of regex will change f

,user_followers,date,text,hashtags
1999534,511.0,2022-01-14 23:59:59,death cross dump,['bitcoin']
1999535,26.0,2022-01-14 23:59:56,is revolutionizing the insurance industry,['bitcoin']
1999536,94.0,2022-01-14 23:59:56,teaser,"['Bitcoin', 'cryptocurrency', 'furniture']"
1999537,51.0,2022-01-14 23:59:56,i am claiming my free lightning sats from s a...,"['Bitcoin', 'BTC', 'LN', 'LightningNetwork', '..."
1999541,7624.0,2022-01-14 23:59:44,did as we expected btc,"['SCRT', 'BTC', 'BITCOIN']"


In [ ]:
# remove rows where tweet text length is less than 4
df2 = df2[df2['text'].str.split().str.len().gt(3)]  

In [ ]:
df2.shape

(474435, 4)

In [ ]:
df2.drop(['hashtags'],axis=1,inplace=True)

In [4]:
# install libraries in colab
!pip install transformers
! pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.3 MB/s 


In [2]:
#df2 = pd.read_csv("clean_data_may22.csv")

In [5]:
# get VADER sentiment scores
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert",num_labels=3)
nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

In [6]:
analyzer = SentimentIntensityAnalyzer()

In [7]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
 
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def analyze_sentiment_pos(tweet):
    sentiment_dict = analyzer.polarity_scores(tweet)
    return str(sentiment_dict['pos'])

def analyze_sentiment_Neg(tweet):
    sentiment_dict = analyzer.polarity_scores(tweet)
    return sentiment_dict['neg']

def analyze_sentiment_Neu(tweet):
    sentiment_dict = analyzer.polarity_scores(tweet)
    return sentiment_dict['neu']

def analyze_sentiment_Comp(tweet):
    sentiment_dict = analyzer.polarity_scores(tweet)
    return sentiment_dict['compound']

def analyze_sentiment_BERT_int(tweet):
    sent = 0
    sentiment_dict = nlp(tweet)
    
    if sentiment_dict[0]['label']=='positive':
      sent = 2
    elif sentiment_dict[0]['label']=='neutral':
      sent = 1
    else: sent = -1
    return sentiment_dict[0]['score'], sent

In [8]:
df2["text"]  = df2["text"].astype(str) 
df2["Vader_Comp"] = df2['text'].apply(analyze_sentiment_Comp)
df2["Vader_Pos"] = df2['text'].apply(analyze_sentiment_pos)
df2["Vader_Neg"] = df2['text'].apply(analyze_sentiment_Neg)
df2["Vader_Neu"] = df2['text'].apply(analyze_sentiment_Neu)


In [9]:
df2.to_csv("newDataset_apr21.csv")

In [10]:
df2.head()
#ran out of memory

,Unnamed: 0,user_followers,date,text,Vader_Comp,Vader_Pos,Vader_Neg,Vader_Neu
0,3028956,102.0,2022-05-02 23:59:54,the future whitelabel solution for pro sp...,0.3182,0.119,0.0,0.881
1,3028957,1095.0,2022-05-02 23:59:54,100 cash or a,0.0000,0.0,0.0,1.000
2,3028958,844.0,2022-05-02 23:59:54,the way i see it just like is likely to trip...,0.1901,0.03,0.0,0.970
3,3028959,102.0,2022-05-02 23:59:47,the future whitelabel solution for pro ...,0.3182,0.119,0.0,0.881
4,3028965,255.0,2022-05-02 23:59:18,current price 38 528 09,0.0000,0.0,0.0,1.000


In [ ]:
df2["BERT_SCORE"], df2["BERT_LABEL"] = df2['text'].apply(analyze_sentiment_BERT_int)

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.to_csv("data_bert_apr21.csv")

In [3]:
df_bert = pd.read_csv("bert_data_apr21.csv")
df_bert.date = pd.to_datetime(df_bert.date)
df_bert.date = df_bert.date.dt.date

In [4]:
df_bert.head()

,Unnamed: 0.1,Unnamed: 0,user_followers,date,text,Vader_Comp,Vader_Pos,Vader_Neg,Vader_Neu,BERT_SCORE,BERT_LABEL
0,0,48583,880.0,2021-04-12,the why behind microsoft s 19 billion nuance ...,0.0000,0.0,0.000,1.000,0.921764,1
1,1,48585,411.0,2021-04-12,what are the biggest shitcoins,0.0000,0.0,0.000,1.000,0.814330,1
2,2,48587,327.0,2021-04-12,market is weakly trending up current moment...,-0.4215,0.0,0.203,0.797,0.707965,2
3,3,48589,61.0,2021-04-12,time to get in folks massive bull run is sta...,0.0000,0.0,0.000,1.000,0.791150,1
4,4,48590,1143.0,2021-04-12,ewt btc eth dot ada link snx inj band...,-0.3400,0.0,0.156,0.844,0.900869,1


In [5]:
cond = [df_bert["BERT_LABEL"] == 2, df_bert["BERT_LABEL"] == 1]
ch = [1,0]
df_bert["bert_label_new"] = np.select(cond, ch, -1)

In [6]:
cond = [df_bert["Vader_Comp"] > 0, df_bert["Vader_Comp"] == 0]
ch = [1,0]
df_bert["vader_label_new"] = np.select(cond, ch, -1)

In [7]:
df_bert.head()

,Unnamed: 0.1,Unnamed: 0,user_followers,date,text,Vader_Comp,Vader_Pos,Vader_Neg,Vader_Neu,BERT_SCORE,BERT_LABEL,bert_label_new,vader_label_new
0,0,48583,880.0,2021-04-12,the why behind microsoft s 19 billion nuance ...,0.0000,0.0,0.000,1.000,0.921764,1,0,0
1,1,48585,411.0,2021-04-12,what are the biggest shitcoins,0.0000,0.0,0.000,1.000,0.814330,1,0,0
2,2,48587,327.0,2021-04-12,market is weakly trending up current moment...,-0.4215,0.0,0.203,0.797,0.707965,2,1,-1
3,3,48589,61.0,2021-04-12,time to get in folks massive bull run is sta...,0.0000,0.0,0.000,1.000,0.791150,1,0,0
4,4,48590,1143.0,2021-04-12,ewt btc eth dot ada link snx inj band...,-0.3400,0.0,0.156,0.844,0.900869,1,0,-1


In [8]:
grouped = df_bert.groupby("date").agg(bert_label=('bert_label_new', 'mean'), 
                               vader_label=('vader_label_new', 'mean'))

In [9]:
grouped

,bert_label,vader_label
date,,
2021-04-05,0.085898,0.305655
2021-04-06,0.062600,0.253210
2021-04-07,0.015861,0.190332
2021-04-08,0.030841,0.213350
2021-04-09,0.046340,0.194890
2021-04-10,0.078589,0.267041
2021-04-11,0.035573,0.285375
2021-04-12,0.034625,0.253916
2021-04-17,0.007220,0.389892


In [10]:
grouped.agg(["min", "max"])

,bert_label,vader_label
min,-0.043496,0.190332
max,0.085898,0.389892
